## Multi-objective unit commitment model, inspired by _**[Power Systems Optimization](https://github.com/east-winds/power-systems-optimization)**_

Andrew Dircks (abd93@cornell.edu)

In [1]:
using JuMP, GLPK
using Plots; plotly();
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
using Printf 
include("vOptGeneric.jl/src/vOptGeneric.jl")  # fork here: https://github.com/andrewdircks/vOptGeneric.jl.git
ENV["COLUMNS"]=120 # Set so all columns of DataFrames and Matrices are displayed

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1278
┌ Warning: Module MbedTLS with build ID 66868493458349 is missing from the cache.
│ This may mean MbedTLS [739be429-bea8-5141-9913-cc70e7f3736d] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1017
┌ Warning: Module MbedTLS with build ID 66868493458349 is missing from the cache.
│ This may mean MbedTLS [739be429-bea8-5141-9913-cc70e7f3736d] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1017
┌ Warning: Module MbedTLS with build ID 66868493458349 is missing from the cache.
│ This may mean MbedTLS [739be429-bea8-5141-9913-cc70e7f3736d] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1017
┌ Warning: Module MbedTLS with build ID 66868493458349 is missing from the cache.
│ This may mean MbedTLS [739be429-bea8-5141-9913-cc70e7f3736d] does not support precompilation but is imp

120

**Load data**

In [2]:
datadir = joinpath("uc_data") 
gen_info = CSV.read(joinpath(datadir,"Generators_data.csv"), DataFrame);
fuels = CSV.read(joinpath(datadir,"Fuels_data.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"Demand.csv"), DataFrame);
gen_variable = CSV.read(joinpath(datadir,"Generators_variability.csv"), DataFrame);

# Rename all columns to lowercase (by convention)
for f in [gen_info, fuels, loads, gen_variable]
    rename!(f,lowercase.(names(f)))
end

**Construct generator dataframe**

In [3]:
# Keep columns relevant to our UC model 
select!(gen_info, 1:26) # columns 1:26
gen_df = outerjoin(gen_info,  fuels, on = :fuel) # load in fuel costs and add to data frame
rename!(gen_df, :cost_per_mmbtu => :fuel_cost)   # rename column for fuel cost
gen_df.fuel_cost[ismissing.(gen_df[:,:fuel_cost])] .= 0

# create "is_variable" column to indicate if this is a variable generation source (e.g. wind, solar):
gen_df[!, :is_variable] .= false
gen_df[in(["onshore_wind_turbine","small_hydroelectric","solar_photovoltaic"]).(gen_df.resource),
    :is_variable] .= true;

# create full name of generator (including geographic location and cluster number)
#  for use with variable generation dataframe
gen_df.gen_full = lowercase.(gen_df.region .* "_" .* gen_df.resource .* "_" .* string.(gen_df.cluster) .* ".0");

# remove generators with no capacity (e.g. new build options that we'd use if this was capacity expansion problem)
gen_df = gen_df[gen_df.existing_cap_mw .> 0,:];
gen_df[!, :start_cost_per_mw] ./ 2;

**Modify load and variable generation dataframes**

In [4]:
# 1. Convert from GMT to GMT-8
gen_variable.hour = mod.(gen_variable.hour .- 9, 8760) .+ 1 
sort!(gen_variable, :hour)
loads.hour = mod.(loads.hour .- 9, 8760) .+ 1
sort!(loads, :hour);

# 2. Convert from "wide" to "long" format
gen_variable_long = stack(gen_variable, 
                        Not(:hour), 
                        variable_name=:gen_full,
                        value_name=:cf);
# Now we have a "long" dataframe; 

In [5]:
#=
Function to convert JuMP outputs (technically, AxisArrays) with two-indexes to a dataframe
Inputs:
    var -- JuMP AxisArray (e.g., value.(GEN))
Reference: https://jump.dev/JuMP.jl/v0.19/containers/
=#
function value_to_df_2dim(var)
    solution = DataFrame(var.data)
    ax1 = var.axes[1]
    ax2 = var.axes[2]
    cols = names(solution)
    insertcols!(solution, 1, :r_id => ax1)
    solution = stack(solution, Not(:r_id), variable_name=:hour)
    solution.hour = foldl(replace, [cols[i] => ax2[i] for i in 1:length(ax2)], init=solution.hour)
    rename!(solution, :value => :gen)
    solution.hour = convert.(Int64,solution.hour)
    return solution
end

value_to_df_2dim (generic function with 1 method)

Then we'll create a function to create and solve a simple unit commitment problem with specified input data.

In [6]:
#=
Function to solve multi-objective unit commitment problem (commitment equations)
Inputs:
    gen_df -- dataframe with generator info
    loads  -- load by time
    gen_variable -- capacity factors of variable generators (in "long" format)
=#
function unit_commitment_multi(gen_df, loads, gen_variable, runtime)
    
    # multi-objective formulation
    UC = Main.vOptGeneric.vModel(GLPK.Optimizer); JuMP.set_silent(UC)

    
    # We reduce the MIP gap tolerance threshold here to increase tractability
    # Here we set it to a 1% gap, meaning that we will terminate once we have 
    # a feasible integer solution guaranteed to be within 1% of the objective
    # function value of the optimal solution.
    # Note that GLPK's default MIP gap is 0.0, meaning that it tries to solve
    # the integer problem to optimality, which can take a LONG time for 
    # any complex problem. So it is important to set this to a realistic value.
    set_optimizer_attribute(UC, "mip_gap", 0.1)

    # Define sets based on data
    # Note the creation of several different sets of generators for use in
    # different equations.
        # Thermal resources for which unit commitment constraints apply
    G_thermal = gen_df[gen_df[!,:up_time] .> 0,:r_id] 
        # Non-thermal resources for which unit commitment constraints do NOT apply 
    G_nonthermal = gen_df[gen_df[!,:up_time] .== 0,:r_id]
        # Variable renewable resources
    G_var = gen_df[gen_df[!,:is_variable] .== 1,:r_id]
        # Non-variable (dispatchable) resources
    G_nonvar = gen_df[gen_df[!,:is_variable] .== 0,:r_id]
        # Non-variable and non-thermal resources
    G_nt_nonvar = intersect(G_nonvar, G_nonthermal)
        # Set of all generators (above are all subsets of this)
    G = gen_df.r_id
        # All time periods (hours) over which we are optimizing
    T = loads.hour
        # A subset of time periods that excludes the last time period
    T_red = loads.hour[1:end-1]  # reduced time periods without last one

    # Generator capacity factor time series for variable generators
    gen_var_cf = innerjoin(gen_variable, 
                    gen_df[gen_df.is_variable .== 1 , 
                        [:r_id, :gen_full, :existing_cap_mw]], 
                    on = :gen_full)
        
    # Decision variables   
    @variables(UC, begin
            # Continuous decision variables
        GEN[G, T]  >= 0     # generation
            # Bin = binary variables; 
            # the following are all binary decisions that 
            # can ONLY take the values 0 or 1
            # The presence of these discrete decisions makes this an MILP
        COMMIT[G_thermal, T], Bin # commitment status (Bin=binary)
        START[G_thermal, T], Bin  # startup decision
        SHUT[G_thermal, T], Bin   # shutdown decision
    end)
    
    # emissions objective
    @Main.vOptGeneric.addobjective(UC, Min, sum(gen_df[gen_df.r_id .== i, :co2_content_tons_per_mmbtu][1] * GEN[i,t] 
                        for i in G for t in T))
                
    # Objective function
        # Sum of variable costs + start-up costs for all generators and time periods
    @Main.vOptGeneric.addobjective(UC, Min, 
        sum( (gen_df[gen_df.r_id .== i,:heat_rate_mmbtu_per_mwh][1] * gen_df[gen_df.r_id .== i,:fuel_cost][1] +
            gen_df[gen_df.r_id .== i,:var_om_cost_per_mwh][1]) * GEN[i,t] 
                        for i in G_nonvar for t in T) + 
        sum(gen_df[gen_df.r_id .== i,:var_om_cost_per_mwh][1] * GEN[i,t] 
                        for i in G_var for t in T)  + 
        sum(gen_df[gen_df.r_id .== i,:start_cost_per_mw][1] * 
            gen_df[gen_df.r_id .== i,:existing_cap_mw][1] *
            START[i,t] 
                        for i in G_thermal for t in T)
    )
    
    # Demand balance constraint (supply must = demand in all time periods)
    @constraint(UC, cDemand[t in T], 
        sum(GEN[i,t] for i in G) == loads[loads.hour .== t,:demand][1])

    # Capacity constraints 
      # 1. thermal generators requiring commitment
    @constraint(UC, Cap_thermal_min[i in G_thermal, t in T], 
        GEN[i,t] >= COMMIT[i, t] * gen_df[gen_df.r_id .== i,:existing_cap_mw][1] *
                        gen_df[gen_df.r_id .== i,:min_power][1])
    @constraint(UC, Cap_thermal_max[i in G_thermal, t in T], 
        GEN[i,t] <= COMMIT[i, t] * gen_df[gen_df.r_id .== i,:existing_cap_mw][1])

      # 2. non-variable generation not requiring commitment
    @constraint(UC, Cap_nt_nonvar[i in G_nt_nonvar, t in T], 
        GEN[i,t] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1])
    
      # 3. variable generation, accounting for hourly capacity factor
    @constraint(UC, Cap_var[i in 1:nrow(gen_var_cf)], 
            GEN[gen_var_cf[i,:r_id], gen_var_cf[i,:hour] ] <= 
                        gen_var_cf[i,:cf] *
                        gen_var_cf[i,:existing_cap_mw])
    
    # Unit commitment constraints
      # 1. Minimum up time
    @constraint(UC, Startup[i in G_thermal, t in T],
        COMMIT[i, t] >= sum(START[i, tt] 
                        for tt in intersect(T,
                            (t-gen_df[gen_df.r_id .== i,:up_time][1]):t)))

      # 2. Minimum down time
    @constraint(UC, Shutdown[i in G_thermal, t in T],
        1-COMMIT[i, t] >= sum(SHUT[i, tt] 
                        for tt in intersect(T,
                            (t-gen_df[gen_df.r_id .== i,:down_time][1]):t)))
 
      # 3. Commitment state
    @constraint(UC, CommitmentStatus[i in G_thermal, t in T_red],
        COMMIT[i,t+1] - COMMIT[i,t] == START[i,t+1] - SHUT[i,t+1])
    
    # solve UC model
    Main.vOptGeneric.vSolve(UC, method= :dicho, step= 100., time_in_secs= runtime, verbose= false)
    
    # return
    vd = Main.vOptGeneric.getvOptData(UC)
    return vd

end

unit_commitment_multi (generic function with 1 method)

**Set up problem**

In [7]:
# A spring day
n=100
T_period = (n*24+1):((n+1)*24)

# High solar case: 5000 MW
# increase all renewable capacity for more interesting solutions
gen_df_sens = copy(gen_df)
gen_df_sens[gen_df_sens.resource .== "solar_photovoltaic", :existing_cap_mw] .= 5000
gen_df[gen_df.resource .== "onshore_wind_turbine", :existing_cap_mw] .= 750
gen_df[gen_df.resource .== "small_hydroelectric", :existing_cap_mw] .= 100

loads_multi = loads[in.(loads.hour,Ref(T_period)),:]
gen_variable_multi = gen_variable_long[in.(gen_variable_long.hour,Ref(T_period)),:]
;;

**Solve**

In [8]:
runtime = 60*1
solution = unit_commitment_multi(gen_df_sens, loads_multi, gen_variable_multi, runtime);